Based on sergiolucero's code here: https://gist.github.com/sergiolucero/e0a6dd13494f139acd968e231906aef2
and Bert Carremans' code here: https://towardsdatascience.com/visualizing-air-pollution-with-folium-maps-4ce1a1880677

In [13]:
import pandas as pd
import numpy as np
import folium, branca
from folium.plugins import TimestampedGeoJson
from folium import Popup

In [2]:
pdf = pd.read_json('https://tinyurl.com/covid19-github')

In [3]:
pdf.shape

(4352, 12)

In [4]:
# Convert date object column to datetime
pdf['data_dt'] = pd.to_datetime(pdf['data'], yearfirst = True)

In [5]:
# Find mean location to center map viewing window
location = pdf.describe()[['lat','long']].loc['50%'].values

In [16]:
# Strip out any rows where the number of total cases is zero
pdf_nonz = pdf[pdf.totale_casi>0].copy()

In [17]:
pdf_nonz['Radius'] = np.ceil(10 * pdf_nonz['totale_casi'] / pdf_nonz['totale_casi'].max())

In [18]:
# Create a yellow-to-red colormap for the total cases variable
colormap = branca.colormap.linear.YlOrRd_03.scale(0, pdf_nonz.totale_casi.max())

In [24]:
# Compile the rows into "feature" dictionaries for the TimestampedGeoJson object
features = []

In [25]:
for _, row in pdf_nonz.sort_values('totale_casi').iterrows():
    feature = {
        'type': 'Feature',
        'geometry': {
            'type':'Point',
            'coordinates':[row['long'],row['lat']]
        },
        'properties': {
            'time': row['data_dt'].date().__str__(),
            'style': {'color': colormap(row['totale_casi'])},
            'icon': 'circle',
            'iconstyle':{
                'fillColor': colormap(row['totale_casi']),
                'fillOpacity': 0.8,
                'stroke': 'true',
                'radius': row['Radius']
            }
        }
    }
    features.append(feature)

In [26]:
# Initialize the map object and add the colorbar at the top
fm = folium.Map(location = location, zoom_start=5, tile='stamentoner', width=800, height=400).add_child(colormap)

In [27]:
# Initialize the TimestampedGeoJson object and add to map
TimestampedGeoJson(
    {'type': 'FeatureCollection',
    'features': features}
    , period='P1D'
    , add_last_point=True
    , auto_play=False
    , loop=False
    , max_speed=1
    , loop_button=True
    , date_options='YYYY/MM/DD'
    , time_slider_drag_update=True
    , duration='P1D'
).add_to(fm)

In [28]:
fm